In [1]:
import numpy as np
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

import pydantic as pd
from typing import Tuple

from data import FieldDataPlotly
import sys
sys.path.append('../../../')
from tidy3d import SimulationData

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  after removing the cwd from sys.path.


In [52]:
class App:
    
    def __init__(self, sim_data : SimulationData):
        self.sim_data = sim_data
        self.field_data = FieldDataPlotly(data=sim_data['fields'])
        self.field_val = list(self.field_data.data.data_dict.keys())[0]
        self.cs_axis = 2
        
        fields = list(self.field_data.data.data_dict.keys())
        freqs = self.field_data.data[self.field_val].f.values.tolist()
        cs_vals = self.field_data.data[self.field_val].coords['xyz'[self.cs_axis]].values.tolist()
            
        self.f_val = freqs[0]
        self.cs_val = np.mean(cs_vals)
        self.val_val = 'real'
        self.app = self.make_app()
        
    def plot_field(self):
        xyz_kwargs=dict(x=None, y=None, z=None)
        xyz_kwargs["xyz"[self.cs_axis]] = self.cs_val
        fig = self.field_data.plotly(
            field=self.field_val,
            freq=self.f_val,
            val=self.val_val,
            **xyz_kwargs
        )
        return fig
        
    def make_app(self):

        app = JupyterDash(__name__)
        app.layout = html.Div(
            [
                html.Div(
                    [
                        html.H1(f'Viewing data for FieldMonitor: "field"'),
                        html.H2(f'Field component.'),
                        dcc.Dropdown(
                            options=list(self.field_data.data.data_dict.keys()),
                            value=self.field_val,
                            id='field_dropdown',
                        ),
                        html.H2(f'Value to plot.'),
                        dcc.Dropdown(
                            options=['real', 'imag', 'abs'],
                            value=self.val_val,
                            id='val_dropdown',
                        ),            
                        html.Br(),
                        html.Div(
                            [
                                dcc.Dropdown(
                                    options=['x', 'y', 'z'],
                                    value='xyz'[self.cs_axis],
                                    id='cs_axis_dropdown',
                                ),
                                dcc.Slider(
                                    min=self.field_data.data[self.field_val].coords['xyz'[self.cs_axis]].values.tolist()[0],
                                    max=self.field_data.data[self.field_val].coords['xyz'[self.cs_axis]].values.tolist()[-1],
                                    value=self.cs_val,
                                    id='cs_slider',
                                ),
                            ],# style={'display': 'flex', 'flex':0,  'padding': 5, 'flex-direction': 'row'}
                        )
                    ], style={'padding': 10, 'flex': 1}
                ),
                html.Div(
                    [
                        dcc.Graph(
                            id='field_plot',
                            figure=self.plot_field(),
                        )
                    ], style={'padding': 10, 'flex': 1}
                )
            ], style={'display': 'flex', 'flex-direction': 'row'}
        )

        @app.callback(
            Output('field_plot', 'figure'),
            [
                Input('field_dropdown', 'value'),
                Input('val_dropdown', 'value'),
                Input('cs_axis_dropdown', 'value'),
                Input('cs_slider', 'value'),
                #Input('f_slider', 'value')
            ]
        )
        def set_field(value_field, value_val, value_cs_axis, value_cs):#, value_f):
            self.field_val = str(value_field)
            self.val_val = str(value_val)
            self.cs_axis = ['x', 'y', 'z'].index(value_cs_axis)
            self.cs_val = float(value_cs)
            
            fig = self.plot_field()
            return fig
                  
        return app
    
    def run(self):
    
        self.app.run_server(mode='jupyterlab', port = 8090, dev_tools_ui=True, #debug=True,
              dev_tools_hot_reload =True, threaded=True)

In [53]:
sim_data = SimulationData.from_file('data/box_full.hdf5')
app = App(sim_data=sim_data)

In [54]:
app.run()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/jupyter_dash/jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.



In [28]:
f0 = float(sim_data['field'].Ex.f.values)

[13:33:10] ERROR    monitor 'field' not found                                       ]8;id=582760;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/log.py\log.py]8;;\:]8;id=598823;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/log.py#33\33]8;;\

DataError: (DataError(...), "monitor 'field' not found")

In [ ]:
dpy = FieldDataPlotly(data=sim_data['field'])

In [24]:
fig = dpy.plotly(freq=f0, val='re', field='Ex', z=0.1)
fig.show()

NameError: name 'dpy' is not defined

In [ ]:
fig = dpy.plotly(freq=f0, val='re', field='Ex')

In [ ]:
fig.show()

In [ ]:
sim_data['field'].Ey.isel(f=0, z=1).real.plot.pcolormesh(x='x', y='y')

In [ ]:
sim_data['field'].Ex.z

In [ ]:
list(sim_data['field'].data_dict.keys())